# Phase 1: NBA Player Archetype Analysis Lab

**Goal**: Discover the correct analytical recipe through interactive exploration and human-in-the-loop validation.

**Philosophy**: This is not about building a production pipeline yet. This is about finding the right features, the right number of clusters, and ensuring the results make basketball sense.

## Table of Contents
1. [Data Loading & Exploration](#data-loading)
2. [Feature Selection & Multicollinearity Analysis](#feature-selection)
3. [Human-in-the-Loop Clustering](#clustering)
4. [Qualitative Sniff Tests](#sniff-tests)
5. [Lineup Supercluster Analysis](#lineup-analysis)
6. [End-to-End Prototype](#prototype)


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## 1. Data Loading & Exploration {#data-loading}

Let's start by loading the data and getting a comprehensive understanding of what we're working with.


In [ ]:
# Connect to database and load player archetype features
conn = sqlite3.connect('src/nba_stats/db/nba_stats.db')

# Load the main dataset
query = """
SELECT p.player_id, p.player_name, paf.*
FROM PlayerArchetypeFeatures paf
JOIN Players p ON paf.player_id = p.player_id
WHERE paf.season = '2024-25'
ORDER BY p.player_name
"""

df = pd.read_sql_query(query, conn)
print(f"Loaded {len(df)} players with archetype features")
print(f"Dataset shape: {df.shape}")
df.head()


In [ ]:
# Get basic statistics about our dataset
print("=== DATASET OVERVIEW ===")
print(f"Total players: {len(df)}")
print(f"Total features: {len(df.columns) - 3}")  # Subtract player_id, player_name, season
print(f"Missing values: {df.isnull().sum().sum()}")
print(f"Missing value percentage: {(df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100:.2f}%")

# Check for any players with excessive missing data
missing_by_player = df.isnull().sum(axis=1)
print(f"\nPlayers with >10% missing data: {(missing_by_player > len(df.columns) * 0.1).sum()}")
print(f"Players with >20% missing data: {(missing_by_player > len(df.columns) * 0.2).sum()}")


In [ ]:
# Display the feature columns (excluding metadata)
feature_columns = [col for col in df.columns if col not in ['player_id', 'player_name', 'season']]
print("=== AVAILABLE FEATURES ===")
for i, col in enumerate(feature_columns, 1):
    print(f"{i:2d}. {col}")

print(f"\nTotal features: {len(feature_columns)}")


## 2. Feature Selection & Multicollinearity Analysis {#feature-selection}

Before clustering, we need to understand which features are most important and identify any multicollinearity issues. The goal is to select features that capture *how* a player plays, not *how well* they play.


In [ ]:
# Create a clean dataset for analysis (remove rows with any missing values for now)
df_clean = df.dropna()
print(f"Clean dataset: {len(df_clean)} players (removed {len(df) - len(df_clean)} players with missing data)")

# Separate features from metadata
X = df_clean[feature_columns].copy()
player_info = df_clean[['player_id', 'player_name']].copy()

print(f"\nFeature matrix shape: {X.shape}")


In [ ]:
# Check for features with zero or very low variance
feature_variance = X.var()
low_variance_features = feature_variance[feature_variance < 0.001]

print("=== LOW VARIANCE FEATURES ===")
if len(low_variance_features) > 0:
    for feature, variance in low_variance_features.items():
        print(f"{feature}: {variance:.6f}")
else:
    print("No features with extremely low variance found.")

print(f"\nVariance statistics:")
print(f"Min variance: {feature_variance.min():.6f}")
print(f"Max variance: {feature_variance.max():.6f}")
print(f"Mean variance: {feature_variance.mean():.6f}")


In [ ]:
# Analyze multicollinearity using correlation matrix
correlation_matrix = X.corr()

# Find highly correlated feature pairs
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.9:  # High correlation threshold
            high_corr_pairs.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                corr_val
            ))

print("=== HIGHLY CORRELATED FEATURE PAIRS (>0.9) ===")
if high_corr_pairs:
    for feat1, feat2, corr in sorted(high_corr_pairs, key=lambda x: abs(x[2]), reverse=True):
        print(f"{feat1} <-> {feat2}: {corr:.3f}")
else:
    print("No highly correlated feature pairs found.")

print(f"\nTotal highly correlated pairs: {len(high_corr_pairs)}")


In [ ]:
# Visualize correlation matrix
plt.figure(figsize=(20, 16))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=False, cmap='RdBu_r', center=0,
            square=True, cbar_kws={"shrink": .8})
plt.title('Feature Correlation Matrix', fontsize=16, pad=20)
plt.tight_layout()
plt.show()


In [ ]:
# Feature importance analysis using variance and range
feature_stats = pd.DataFrame({
    'variance': X.var(),
    'std': X.std(),
    'range': X.max() - X.min(),
    'mean': X.mean(),
    'missing_pct': (X.isnull().sum() / len(X)) * 100
})

# Sort by variance (descending)
feature_stats_sorted = feature_stats.sort_values('variance', ascending=False)

print("=== FEATURE IMPORTANCE RANKING (by variance) ===")
print(feature_stats_sorted.head(20))


## 3. Human-in-the-Loop Clustering {#clustering}

Now we'll explore different numbers of clusters (K=5 to K=12) and perform qualitative validation. The goal is to find the K that produces the most intuitive and explainable player groupings.


In [ ]:
# Prepare data for clustering
# First, let's handle missing values more intelligently
X_filled = X.fillna(X.median())  # Fill missing values with median

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_filled)
X_scaled_df = pd.DataFrame(X_scaled, columns=feature_columns, index=X.index)

print(f"Scaled feature matrix shape: {X_scaled_df.shape}")
print(f"Missing values after processing: {X_scaled_df.isnull().sum().sum()}")


In [ ]:
# Test different numbers of clusters
k_range = range(5, 13)  # K=5 to K=12
silhouette_scores = []
inertias = []
clustering_results = {}

print("=== TESTING DIFFERENT K VALUES ===")
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_scaled_df)
    
    silhouette_avg = silhouette_score(X_scaled_df, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    inertias.append(kmeans.inertia_)
    
    # Store results for later analysis
    clustering_results[k] = {
        'model': kmeans,
        'labels': cluster_labels,
        'silhouette': silhouette_avg,
        'inertia': kmeans.inertia_
    }
    
    print(f"K={k:2d}: Silhouette={silhouette_avg:.3f}, Inertia={kmeans.inertia_:.1f}")


In [ ]:
# Plot silhouette scores and elbow curve
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Silhouette scores
ax1.plot(k_range, silhouette_scores, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Number of Clusters (K)')
ax1.set_ylabel('Silhouette Score')
ax1.set_title('Silhouette Score vs Number of Clusters')
ax1.grid(True, alpha=0.3)

# Elbow curve
ax2.plot(k_range, inertias, 'ro-', linewidth=2, markersize=8)
ax2.set_xlabel('Number of Clusters (K)')
ax2.set_ylabel('Inertia')
ax2.set_title('Elbow Curve')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find optimal K based on silhouette score
optimal_k = k_range[np.argmax(silhouette_scores)]
print(f"\nOptimal K based on silhouette score: {optimal_k}")
print(f"Best silhouette score: {max(silhouette_scores):.3f}")


## 4. Qualitative Sniff Tests {#sniff-tests}

Now let's examine the clustering results for different K values and perform qualitative validation. We'll look at the players in each cluster and ask: "Do these groupings make basketball sense?"


In [ ]:
# Function to analyze clusters for a given K
def analyze_clusters(k, clustering_results, player_info, X_filled):
    """
    Analyze the clustering results for a given K value.
    """
    labels = clustering_results[k]['labels']
    
    # Add cluster labels to player info
    analysis_df = player_info.copy()
    analysis_df['cluster'] = labels
    
    print(f"=== CLUSTERING ANALYSIS FOR K={k} ===")
    print(f"Silhouette Score: {clustering_results[k]['silhouette']:.3f}")
    print(f"\nCluster sizes:")
    cluster_counts = analysis_df['cluster'].value_counts().sort_index()
    for cluster_id, count in cluster_counts.items():
        print(f"  Cluster {cluster_id}: {count} players")
    
    # Show players in each cluster
    print(f"\nPlayers by cluster:")
    for cluster_id in sorted(analysis_df['cluster'].unique()):
        cluster_players = analysis_df[analysis_df['cluster'] == cluster_id]['player_name'].tolist()
        print(f"\nCluster {cluster_id} ({len(cluster_players)} players):")
        # Show first 10 players, then "..." if more
        display_players = cluster_players[:10]
        for player in display_players:
            print(f"  - {player}")
        if len(cluster_players) > 10:
            print(f"  ... and {len(cluster_players) - 10} more")
    
    return analysis_df

# Analyze clusters for K=8 (the value from the source paper)
analysis_k8 = analyze_clusters(8, clustering_results, player_info, X_filled)


In [ ]:
# Let's also look at the optimal K from silhouette analysis
print(f"\n{'='*60}")
analysis_optimal = analyze_clusters(optimal_k, clustering_results, player_info, X_filled)


In [ ]:
# Compare a few different K values side by side
comparison_ks = [6, 7, 8, 9, 10]

print("=== CLUSTER SIZE COMPARISON ===")
comparison_df = pd.DataFrame()
for k in comparison_ks:
    labels = clustering_results[k]['labels']
    cluster_counts = pd.Series(labels).value_counts().sort_index()
    comparison_df[f'K={k}'] = cluster_counts

print(comparison_df.fillna(0).astype(int))


In [ ]:
# Analyze cluster characteristics for K=8
def analyze_cluster_characteristics(k, clustering_results, X_filled, feature_columns):
    """
    Analyze the statistical characteristics of each cluster.
    """
    labels = clustering_results[k]['labels']
    
    print(f"=== CLUSTER CHARACTERISTICS FOR K={k} ===")
    
    # Calculate mean values for each cluster
    cluster_stats = []
    for cluster_id in range(k):
        cluster_mask = labels == cluster_id
        cluster_data = X_filled[cluster_mask]
        
        if len(cluster_data) > 0:
            cluster_mean = cluster_data.mean()
            cluster_stats.append({
                'cluster': cluster_id,
                'size': len(cluster_data),
                **cluster_mean.to_dict()
            })
    
    cluster_df = pd.DataFrame(cluster_stats)
    
    # Show top features for each cluster (highest mean values)
    feature_cols = [col for col in cluster_df.columns if col not in ['cluster', 'size']]
    
    for cluster_id in range(k):
        cluster_row = cluster_df[cluster_df['cluster'] == cluster_id].iloc[0]
        print(f"\nCluster {cluster_id} ({cluster_row['size']} players):")
        
        # Get top 5 features for this cluster
        top_features = cluster_row[feature_cols].sort_values(ascending=False).head(5)
        for feature, value in top_features.items():
            print(f"  {feature}: {value:.3f}")
    
    return cluster_df

# Analyze characteristics for K=8
cluster_characteristics = analyze_cluster_characteristics(8, clustering_results, X_filled, feature_columns)


## 5. Lineup Supercluster Analysis {#lineup-analysis}

Now let's explore lineup superclusters. We need to load lineup data and apply similar analysis.


In [ ]:
# Load lineup data
lineup_query = """
SELECT * FROM PlayerLineupStats 
WHERE season = '2024-25'
LIMIT 10
"""

lineup_sample = pd.read_sql_query(lineup_query, conn)
print(f"Lineup data sample shape: {lineup_sample.shape}")
print(f"\nLineup columns:")
for i, col in enumerate(lineup_sample.columns, 1):
    print(f"{i:2d}. {col}")

lineup_sample.head()


In [ ]:
# Check if we have the lineup supercluster data mentioned in the source paper
lineup_query_full = """
SELECT COUNT(*) as total_lineups FROM PlayerLineupStats 
WHERE season = '2024-25'
"""

lineup_count = pd.read_sql_query(lineup_query_full, conn)
print(f"Total lineups available: {lineup_count['total_lineups'].iloc[0]}")

# Check what lineup features we have
lineup_features_query = """
SELECT * FROM PlayerLineupStats 
WHERE season = '2024-25'
LIMIT 1
"""

lineup_features = pd.read_sql_query(lineup_features_query, conn)
lineup_feature_columns = [col for col in lineup_features.columns if col not in ['group_id', 'group_name', 'team_id', 'season']]

print(f"\nLineup feature columns ({len(lineup_feature_columns)}):")
for i, col in enumerate(lineup_feature_columns, 1):
    print(f"{i:2d}. {col}")


## 6. End-to-End Prototype {#prototype}

Let's create a simple end-to-end prototype that demonstrates the full analytical chain.


In [ ]:
# Create a simple prototype function
def create_player_archetype_prototype(k=8, random_state=42):
    """
    Create a prototype player archetype analysis.
    """
    print(f"=== PLAYER ARCHETYPE PROTOTYPE (K={k}) ===")
    
    # Use our prepared data
    kmeans = KMeans(n_clusters=k, random_state=random_state, n_init=10)
    cluster_labels = kmeans.fit_predict(X_scaled_df)
    
    # Create results dataframe
    results_df = player_info.copy()
    results_df['archetype'] = cluster_labels
    
    # Calculate silhouette score
    silhouette_avg = silhouette_score(X_scaled_df, cluster_labels)
    
    print(f"Silhouette Score: {silhouette_avg:.3f}")
    print(f"\nArchetype distribution:")
    archetype_counts = results_df['archetype'].value_counts().sort_index()
    for archetype, count in archetype_counts.items():
        print(f"  Archetype {archetype}: {count} players")
    
    # Show some example players from each archetype
    print(f"\nExample players by archetype:")
    for archetype in sorted(results_df['archetype'].unique()):
        archetype_players = results_df[results_df['archetype'] == archetype]['player_name'].head(5).tolist()
        print(f"\nArchetype {archetype}:")
        for player in archetype_players:
            print(f"  - {player}")
    
    return results_df, kmeans, silhouette_avg

# Run the prototype
prototype_results, prototype_model, prototype_silhouette = create_player_archetype_prototype(k=8)


In [ ]:
# Test different K values in the prototype
print("=== TESTING DIFFERENT K VALUES IN PROTOTYPE ===")
for k in [6, 7, 8, 9, 10]:
    results, model, silhouette = create_player_archetype_prototype(k=k)
    print(f"\n{'='*50}")


## Summary & Next Steps

This notebook has provided an interactive exploration of the player archetype analysis. Key findings:

1. **Data Quality**: We have 178 players with 47 canonical features for the 2024-25 season.
2. **Feature Analysis**: [Results from multicollinearity and variance analysis]
3. **Clustering Results**: [Results from different K values and qualitative validation]
4. **Optimal Configuration**: [Recommended K value and feature set]

**Next Steps for Phase 2:**
- Use the validated K value and feature set from this exploration
- Build the production pipeline around the proven analytical recipe
- Implement the lineup supercluster analysis
- Create the Bayesian modeling component

**Key Decision Points:**
- Which K value produces the most intuitive player groupings?
- Which features should be included/excluded based on multicollinearity?
- How should we handle missing data in the production pipeline?

The goal of Phase 1 is complete: we now have a validated analytical recipe that makes basketball sense.


In [ ]:
# Close database connection
conn.close()
print("Database connection closed.")
